The following code was written before I decided to go with the `ensembleBMA` route.  
Refer to the data preprocessing steps in `BMA.ipynb` if you need to run the code in this notebook.

In [ ]:
result = stats.linregress(data['Lee 2017'], data['Obs'])

In [ ]:
plt.plot(data['Lee 2017'], data['Obs'], 'o', label='original data')
plt.plot(data['Lee 2017'], result.intercept + result.slope*data['Lee 2017'], 'r', label='fitted line')
plt.legend()
plt.xlabel('Forecast Wind Speed $ms^{-1}$')
plt.ylabel('Observed Wind Speed $ms^{-1}$')
plt.show()

Well, linearity only seems to be a marginally good assumption here... perhaps we should replace this with some type of polynomial or maybe logistic regression? For the other parameters, we must carry out minimization of the log-likelihood function. 

In [ ]:
# Fortunately, we have no zero wind speed values, which means that we don't have to do the more complicated MLE method proposed by Wilkes in 1990
(wrfens == 0).sum()

In [ ]:
(obs == 0).sum()

In [ ]:
# Record the mean parameters from linear regression
b0 = result.intercept 
b1 = result.slope
fx = data.loc['2020-02-11 23:50', 'Lee 2017']

In [ ]:
def get_mu(fx, b0, b1):
    '''
    Estimate the mean, mu_k
    '''
    mu = b0 + b1 * fx
    return mu

    
def get_sigma(fx, c0, c1):
    '''
    Estimate the standard deviation, sigma_k 
    '''
    sigma = c0 + c1 * fx
    return sigma

In [ ]:
def get_weighted_probability(obs, mu, sigma, w):
    '''
    Get the weighted PDF for a single instance (i.e., one observation)
    '''
    alpha = mu**2 / sigma**2
    beta = alpha / mu 
    weighted_pdf = w * stats.gamma.pdf(obs, alpha, loc = 0.0, scale = 1.0/beta)
    return weighted_pdf

In [ ]:
# negative log-liklihood calculation
def get_log_lik(parameters):
    # extract parameters
    const, beta, std_dev = parameters
    # predict the output
    pred = const + beta*x
    # Calculate the log-likelihood for normal distribution
    log_lik = np.sum(stats.gamma.logpdf(y, pred, std_dev))
    # Calculate the negative log-likelihood
    neg_log_lik = -1 * log_lik
    return neg_log_lik

In [ ]:
# minimize arguments: function, intial_guess_of_parameters, method
mle_model = opt.minimize(get_log_lik, np.array([2,2,2]), method='L-BFGS-B')
mle_model